In [17]:
import gymnasium as gym
import numpy as np
from random import sample
import pandas as pd
from utilities import split_in_blocks, float_to_bits, int_to_bits, read_timestamps, bits_to_int, unsigned_to_signed, bits_to_int_unsigned, bits_to_float
from transform_primitives import delta_of_delta
from compression_primitives import bitmask, trailing_zero
from constants import parameters_dict
import math
import struct
import ammmo_lazy as aml

class CompressionEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Box(low=np.array([0, 0, 0, 0, 0, 0, 0, 0]), high=np.array([3, 5, 5, 5, 7, 1, 1, 5]), shape=(8,),  dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=np.array([0 for i in range(34 * 8)] ), high=np.array([1 for i in range(34 * 8)]), shape=(34 * 8,),  dtype=np.float32)
        self.idx = 0
        self.timeseries = ['grok_asg_anomaly.csv', 'occupancy_t4013.csv','Sunspots.csv', 'monthly-beer-production.csv', 'monthly-housing.csv', 'cpu_utilization.csv', 'art-price.csv', 'Gold.csv', 'Electric_Production.csv', 'daily-temperatures.csv', 'oil.csv', 'rogue_agent_key_updown.csv']
        self.timestamps = []
        self.timestamps_ratio = 0
        self.metrics = []
        self.paramters = parameters_dict
        self.last_2_metric = []
        self.compressed_block = ''
        self.compare_ratio = 0
        self.get_ts()
        

    def get_ts(self):
        my_ts = sample(self.timeseries, 1)
        #print(my_ts[0])
        df = pd.read_csv(my_ts[0])
        self.timestamps = read_timestamps(df)
        self.timestamps_ratio, compression = self.compress_timestamps(self.timestamps)
        self.idx = 0
        
        self.metrics = (df.iloc[:, 1]).astype(np.float64).values
        self.metrics[np.isnan(self.metrics)==True] = 0
        self.timestamps[np.isnan(self.timestamps)==True] = 0
        self.last_2_metric = self.metrics[:2]
        self.metrics = split_in_blocks(self.metrics[2:])

        self.state = self.metrics[0]

    def compress_timestamps(self, timestamps):
        sum = 192
        header = int_to_bits(len(timestamps) - 2)
        data = int_to_bits(timestamps[0]) + int_to_bits(timestamps[1])
        last_2 = [timestamps[0], timestamps[1]]

        for i in range(2, len(timestamps)):
            val = delta_of_delta(last_2[0], last_2[1], timestamps[i])
            if val == 0:
                header += '0'
                sum += 1
            else:
                header += '1'
                sum += 1
                if val > 0:
                    val -= 1
                if -3 <= val <= 4:
                    data += '1'
                    sum += 4
                    data += int_to_bits(val)[-3:]
                elif -31 <= val <= 32:
                    data += '01'
                    sum += 8
                    data += int_to_bits(val)[-6:]
                else:
                    ct_bitmask, compressed_bitmask = bitmask(int_to_bits(val), 0)
                    ct_trailing_zeros, compressed_trailing = trailing_zero(int_to_bits(val))

                    if ct_bitmask < ct_trailing_zeros:
                        data += '001'
                        sum += ct_bitmask + 3
                        data += compressed_bitmask
                    else:
                        data += '000'
                        sum += ct_trailing_zeros + 3
                        data += compressed_trailing
            last_2 = [last_2[1], timestamps[i]]
        
        return (len(timestamps) * 64 / sum), header + data

    def decompress_timestamps(self, compressed):
        size = int(compressed[:64], 2)
        header = compressed[64:64+size]
        data = compressed[64+size:]

        orig_timestamps = []

        orig_timestamps.append(int(data[:64], 2))
        orig_timestamps.append(int(data[64:128], 2))

        last_2 = [orig_timestamps[0], orig_timestamps[1]]

        data_idx = 128

        for x in header:
            if x == '0':
                orig_timestamps.append(2 * last_2[1] - last_2[0])
            else:
                if data[data_idx] == '1':
                    val = bits_to_int(data[data_idx+1:data_idx+4])
                    if val >= 0:
                        val += 1
                    orig_timestamps.append(val + 2 * last_2[1] - last_2[0])
                    data_idx += 4
                else:
                    if data[data_idx+1] == '1':
                        val = bits_to_int(data[data_idx+2:data_idx+8])
                        if val >= 0:
                            val += 1
                        orig_timestamps.append(val + 2 * last_2[1] - last_2[0])
                        data_idx += 8
                    else:
                        val = 0
                        if data[data_idx+2] == '1':
                            last_bit = data_idx + 11
                            mask = data[data_idx+3:data_idx+11]
                            for bit in mask:
                                if bit == '0':
                                    val <<= 8
                                else:
                                    crt_byte = data[last_bit:last_bit+8]
                                    val = (val << 8) + int(crt_byte, 2)
                                    last_bit += 8
                            data_idx = last_bit + 8
                        else:
                            ct_zero_bytes = int(data[data_idx+3:data_idx+6], 2)
                            ct_non_zero_bytes = int(data[data_idx+6:data_idx+9], 2) + 1

                            for i in range(ct_non_zero_bytes):
                                crt_byte = data[data_idx+9+i*8:data_idx+9+(i+1)*8]
                                val = (val << 8) + int(crt_byte, 2)
                            val = (val << 8 * ct_zero_bytes)
                            data_idx += 9 + ct_non_zero_bytes * 8
                        val = unsigned_to_signed(val)
                        if val >= 0:
                            val += 1
                        orig_timestamps.append(val + 2 * last_2[1] - last_2[0])
                            
            last_2 = [last_2[1], orig_timestamps[-1]]
        return orig_timestamps


    def bytes(self, vec):
        res = []
        for x in vec:
            y = float_to_bits(x)
            for start in range(0, 64, 8):
                byte = y[start:start+8]
                byte = int(byte, 2)
                res.append(byte / 255)
        return np.asarray(res).astype(np.float32)

    def decompress_metrics(self, compressed_block):
        params = compressed_block[:19]
        header = compressed_block[19:51]
        majorMode = bits_to_int_unsigned(params[:2])
        transTypes = [self.paramters[4][bits_to_int_unsigned(params[2:5])], self.paramters[4][bits_to_int_unsigned(params[5:8])], self.paramters[4][bits_to_int_unsigned(params[8:11])]]
        encodings = {
            '00': 0,
            '01': 1,
            '10': 2,
            '11': 3
        }
        possibilites = self.paramters[3][majorMode]
        offByteShift1 = bits_to_int_unsigned(params[11:14])
        offByteShift2 = bits_to_int_unsigned(params[14:15])
        offByteShift3 = bits_to_int_unsigned(params[15:16])
        offBitmask = bits_to_int_unsigned(params[16:19])
        proper_block = compressed_block[51:]

        orig_block = []
        prev = self.last_2_metric
        step = 0

        for x in header:
            if x == '0':
                orig_block.append(prev[1])
                prev = [prev[1], orig_block[-1]]
            else:
                encoding = proper_block[step:step+2]
                idx = encodings[encoding]
                step += 2
                transIdx, compresser, mode = possibilites[idx]

                if compresser.__name__ == 'bitmask':
                    mask = proper_block[step: step + 8 - offBitmask]
                    val = ''
                    step = step + 8 - offBitmask
                    for j in mask:
                        if j == '0':
                            val += '00000000'
                        else:
                            val += proper_block[step:step + 8]
                            step += 8
                    val += (offBitmask * 8) * '0'
                    val = bits_to_float(val)
                    orig_block.append(transTypes[transIdx](prev[0], prev[1], val))
                elif compresser.__name__ == 'offset':
                    offset = offByteShift1
                    match mode:
                        case 1:
                            offset = offByteShift1
                        case 2:
                            offset = offByteShift1 - offByteShift2
                        case 3:
                            offset = offByteShift1 - offByteShift2 - offByteShift3
                    leading_zeros = 64 - offset * 8 - mode * 7
                    val = '0' * leading_zeros + proper_block[step:step+mode * 7] + '0' * (offset * 8)
                    value = bits_to_float(val)
                    value = transTypes[transIdx](prev[0], prev[1], value)
                    orig_block.append(value)
                    step = step + mode * 7
                else:
                    if proper_block[step:step+4] == '1111':
                        val = transTypes[transIdx](prev[0], prev[1], 0.0)
                        step = step + 4
                        orig_block.append(val)
                    else:
                        ct_zero_bytes = int(proper_block[step:step+3], 2)
                        ct_non_zero_bytes = int(proper_block[step+3:step+6], 2) + 1

                        val = 8 * (8 - ct_zero_bytes - ct_non_zero_bytes) * '0'

                        for j in range(ct_non_zero_bytes):
                            crt_byte = proper_block[step+6+j*8:step+6+(j+1)*8]
                            val += crt_byte
                        val += ct_zero_bytes * 8 * '0'
                        step += 6 + ct_non_zero_bytes * 8
                        val = bits_to_float(val)
                        val = transTypes[transIdx](prev[0], prev[1], val)
                        
                        orig_block.append(val)
                
                prev = [prev[1], orig_block[-1]]

        return orig_block


    def step(self, action):
        params = np.round(action).astype(int)
        prev_2 = self.last_2_metric
        self.compare_ratio = len(aml.compress_metrics(np.concatenate([prev_2, self.state]))[128:])
        bits_count = self.compress(params)

        # ###########
        # #decompress
        # self.last_2_metric = prev_2
        # orig_block = self.decompress_metrics(self.compressed_block)

        # for i in range(32):
        #     if orig_block[i] != self.state[i]:
        #         print('Error')
        #         print(orig_block[i], self.state[i])
        # self.last_2_metric = [self.state[-2], self.state[-1]]
        # ###########

        reward = self.evaluate(bits_count)
        self.idx += 1
        if self.idx >= len(self.metrics):
            state = np.pad(self.state, (0, 32 - len(self.state)), 'constant', constant_values=(-1e14, ))
            return self.bytes(np.concatenate([self.last_2_metric, state])), reward, True, False, {'block': self.compressed_block, 'params': self.write_params(action)}

        self.state = self.metrics[self.idx]

        state = np.pad(self.state, (0, 32 - len(self.state)), 'constant', constant_values=(-1e14, ))
        return self.bytes(np.concatenate([self.last_2_metric, state])), reward, False, False, {'block': self.compressed_block, 'params': self.write_params(action)}

    def write_params(self, action):
        params = np.round(action).astype(int)
        params_bits = ''
        params_bits += int_to_bits(params[0])[-2:]
        for i in range(4):
            params_bits += int_to_bits(params[i+1])[-3:]
        params_bits += int_to_bits(params[5])[-1:]
        params_bits += int_to_bits(params[6])[-1:]
        params_bits += int_to_bits(params[7])[-3:]
        return params_bits

    def reset(self, seed=None):
        self.get_ts()
        return self.bytes(np.concatenate([self.last_2_metric, self.state])), {}

    def evaluate(self, compressed):
        val = compressed / self.compare_ratio
        if val >= 1.5:
            return -1
        return -math.tan(val) + 1.5

    def compress(self, params):
        self.compressed_block = ''
        header = self.write_params(params)
        majorMode = params[0]
        transTypes = [self.paramters[1][params[1]], self.paramters[1][params[2]], self.paramters[1][params[3]]]
        offByteShift1 = params[4]
        offByteShift2 = params[5]
        offByteShift3 = params[6]
        offBitmask = params[7] 
        #endian = params[10]
        posibilities = self.paramters[3][majorMode]
        encodings = ['00', '01', '10', '11']

        for i in range(len(self.state)):
            nr = self.state[i]
            opt = 100
            opt_compressed = ''
            idx_op = -1

            last_2 = self.last_2_metric
            val = 0
            if nr == last_2[1]:
                header += '0'
                self.last_2_metric = [last_2[1], nr]
                continue
                
            header += '1'

            for (j, (transformer, compresser, param)) in  enumerate(posibilities):
                val = transTypes[transformer](last_2[0], last_2[1], nr)
                val = float_to_bits(val)
                if compresser.__name__ == 'offset' and param == 1:
                    ct_bits, compressed = compresser(val, offByteShift1, 1)
                elif compresser.__name__ == 'offset' and param == 2:
                    ct_bits, compressed = compresser(val, offByteShift1 - offByteShift2, 2)
                elif compresser.__name__ == 'offset' and param == 3:
                    ct_bits, compressed = compresser(val, offByteShift1 - offByteShift2 - offByteShift3, 3)
                elif compresser.__name__ == 'bitmask':
                    ct_bits, compressed = compresser(val, offBitmask)
                elif compresser.__name__ == 'trailing_zero':
                    ct_bits, compressed = compresser(val)
                    
                if ct_bits < opt:
                    opt = ct_bits
                    opt_compressed = compressed
                    idx_op = j
                        
            self.last_2_metric = [last_2[1], self.state[i]]
            self.compressed_block += encodings[idx_op] + opt_compressed
        self.compressed_block = header + self.compressed_block
        return len(self.compressed_block)


    def render(self):
        #useless
        pass

    def close(self):
        #useless
        pass

In [18]:
env = CompressionEnv()
action = env.action_space.sample()
print(action)
state, reward, done, truncated, info = env.step(action)
sum = reward
ct = 1
maxi = 0
while not done:
    action = env.action_space.sample()
    state, reward, done, truncarted, info = env.step(action)
    #print(reward, action, state)
    # if env.idx >= 1000:
    #     done = True
#     maxi = max(maxi, reward)
#     sum += reward
#     ct += 1
print(sum / ct, maxi, ct)

[2.63820097 3.37790898 2.37372801 0.88547841 4.60483377 0.16547556
 0.91405193 2.22595209]
-0.7899691341197879 0 1


In [19]:
from stable_baselines3.common.env_checker import check_env
env = CompressionEnv()
obs, _ = env.reset()
check_env(env)

/home/dimi/.local/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/home/dimi/.local/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:464: UserWarning: Your action space has dtype float64, we recommend using np.float32 to avoid cast errors.
  warnings.warn(


In [20]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [21]:
#Use SAC from stable baselines
from stable_baselines3 import TD3, DDPG
from stable_baselines3.sac.policies import MlpPolicy, MultiInputPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

env = CompressionEnv()
env = DummyVecEnv([lambda: env])

n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

#Add progress bar for learning
model = DDPG('MlpPolicy', env, action_noise=action_noise, verbose=1)
model3 = model.learn(total_timesteps=50000, progress_bar=True, log_interval=20)


Output()

Using cuda device


---------------------------------
| time/              |          |
|    episodes        | 20       |
|    fps             | 75       |
|    time_elapsed    | 20       |
|    total_timesteps | 1584     |
| train/             |          |
|    actor_loss      | 5.38     |
|    critic_loss     | 0.0855   |
|    learning_rate   | 0.001    |
|    n_updates       | 1483     |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 40       |
|    fps             | 71       |
|    time_elapsed    | 49       |
|    total_timesteps | 3562     |
| train/             |          |
|    actor_loss      | 7.96     |
|    critic_loss     | 0.341    |
|    learning_rate   | 0.001    |
|    n_updates       | 3461     |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 60       |
|    fps             | 70       |
|    time_elapsed    | 75       |
|    total_timesteps | 5330     |
| train/             |          |
|    actor_loss      | 7.96     |
|    critic_loss     | 0.431    |
|    learning_rate   | 0.001    |
|    n_updates       | 5229     |
---------------------------------


...

---------------------------------
| time/              |          |
|    episodes        | 480      |
|    fps             | 69       |
|    time_elapsed    | 587      |
|    total_timesteps | 40640    |
| train/             |          |
|    actor_loss      | 7.76     |
|    critic_loss     | 0.567    |
|    learning_rate   | 0.001    |
|    n_updates       | 40539    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 500      |
|    fps             | 69       |
|    time_elapsed    | 612      |
|    total_timesteps | 42415    |
| train/             |          |
|    actor_loss      | 5.02     |
|    critic_loss     | 0.371    |
|    learning_rate   | 0.001    |
|    n_updates       | 42314    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 520      |
|    fps             | 69       |
|    time_elapsed    | 638      |
|    total_timesteps | 44188    |
| train/             |          |
|    actor_loss      | 2.34     |
|    critic_loss     | 3.35     |
|    learning_rate   | 0.001    |
|    n_updates       | 44087    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 540      |
|    fps             | 69       |
|    time_elapsed    | 668      |
|    total_timesteps | 46327    |
| train/             |          |
|    actor_loss      | -1.01    |
|    critic_loss     | 0.7      |
|    learning_rate   | 0.001    |
|    n_updates       | 46226    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 560      |
|    fps             | 69       |
|    time_elapsed    | 699      |
|    total_timesteps | 48460    |
| train/             |          |
|    actor_loss      | 3.35     |
|    critic_loss     | 8.36     |
|    learning_rate   | 0.001    |
|    n_updates       | 48359    |
---------------------------------


In [23]:
model3 = model.learn(total_timesteps=200000, progress_bar=True, log_interval=80)


Output()

---------------------------------
| time/              |          |
|    episodes        | 80       |
|    fps             | 71       |
|    time_elapsed    | 105      |
|    total_timesteps | 7520     |
| train/             |          |
|    actor_loss      | 0.766    |
|    critic_loss     | 0.105    |
|    learning_rate   | 0.001    |
|    n_updates       | 157219   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 160      |
|    fps             | 70       |
|    time_elapsed    | 202      |
|    total_timesteps | 14316    |
| train/             |          |
|    actor_loss      | 1.73     |
|    critic_loss     | 0.104    |
|    learning_rate   | 0.001    |
|    n_updates       | 164015   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 240      |
|    fps             | 70       |
|    time_elapsed    | 295      |
|    total_timesteps | 20902    |
| train/             |          |
|    actor_loss      | 1.61     |
|    critic_loss     | 0.128    |
|    learning_rate   | 0.001    |
|    n_updates       | 170601   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 320      |
|    fps             | 70       |
|    time_elapsed    | 391      |
|    total_timesteps | 27747    |
| train/             |          |
|    actor_loss      | 1.88     |
|    critic_loss     | 0.0913   |
|    learning_rate   | 0.001    |
|    n_updates       | 177446   |
---------------------------------


...

---------------------------------
| time/              |          |
|    episodes        | 1840     |
|    fps             | 69       |
|    time_elapsed    | 2357     |
|    total_timesteps | 165037   |
| train/             |          |
|    actor_loss      | 0.825    |
|    critic_loss     | 0.0841   |
|    learning_rate   | 0.001    |
|    n_updates       | 314736   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 1920     |
|    fps             | 69       |
|    time_elapsed    | 2457     |
|    total_timesteps | 171957   |
| train/             |          |
|    actor_loss      | 0.684    |
|    critic_loss     | 0.0834   |
|    learning_rate   | 0.001    |
|    n_updates       | 321656   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 2000     |
|    fps             | 69       |
|    time_elapsed    | 2558     |
|    total_timesteps | 179069   |
| train/             |          |
|    actor_loss      | -0.0676  |
|    critic_loss     | 0.0817   |
|    learning_rate   | 0.001    |
|    n_updates       | 328768   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 2080     |
|    fps             | 69       |
|    time_elapsed    | 2662     |
|    total_timesteps | 186315   |
| train/             |          |
|    actor_loss      | -1.38    |
|    critic_loss     | 0.595    |
|    learning_rate   | 0.001    |
|    n_updates       | 336014   |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 2160     |
|    fps             | 69       |
|    time_elapsed    | 2762     |
|    total_timesteps | 193240   |
| train/             |          |
|    actor_loss      | -0.74    |
|    critic_loss     | 0.52     |
|    learning_rate   | 0.001    |
|    n_updates       | 342939   |
---------------------------------


In [38]:
#save the model
model.save("TD3_CompressionEnv")

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [37]:
env = CompressionEnv()
obs, _ = env.reset()

print(env.timestamps_ratio)

action, _states = model.predict(obs)
state, reward, done, truncated, info = env.step(action)
sum = reward
ct = 1
ratio = 128 + len(info['block'])
original = 32 * 64
while not done:
    action, _state = model.predict(state)
    state, reward, done, truncarted, info = env.step(action)
    ratio += len(info['block'])
    original += 32 * 64
    #print(reward)
    sum += reward
    ct += 1
print(original / ratio, env.timestamps_ratio, (original + len(env.timestamps) * 64) / (ratio + len(env.timestamps) * 64 / env.timestamps_ratio) , sum / ct)

61.472458948243606
1.3565640046412615 61.472458948243606 2.6493471662859776 -0.09340688633165718


In [ ]:
from transform_primitives import xor, delta_xor, delta, delta_of_delta, rev_delta, rev_delta_of_delta
from compression_primitives import bitmask, offset, trailing_zero
env = CompressionEnv()
#env.get_ts()
print(env.metrics[0])
sample = env.metrics[0]
prev = env.last_2_metric
print(prev)
transformers = [delta, delta_of_delta, rev_delta, rev_delta_of_delta]
avg = 0
ct = 0
for j in range(len(env.metrics)):
    sample = env.metrics[j]
    for i in range(len(sample)):
        ct += 1
        mini = 64
        for t in transformers:
            val = t(prev[0], prev[1], sample[i])
            val = int(num_to_bits(val), 2)
            mini = min(mini, bitmask(val, 0), offset(val, 0), trailing_zero(val), offset(val, 1), offset(val, 2), bitmask(val, 1))
        avg += mini
        prev[0] = prev[1]
        prev[1] = sample[i]
print(avg / ct)
#daily-temperature 61.26 avg
#gold 61.82

In [ ]:
action = [3, 0, 1, 3, 0, 1, 0, 0, 0, 0]
env2 = CompressionEnv()
print(env2.state)
state, reward, done, truncated, info = env2.step(action)
print(reward)

In [ ]:
df = pd.read_csv('Gold.csv')
x = pd.to_datetime(df['DATE'], format='%Y/%m/%d')
print(x.astype(np.int64))

In [ ]:
from utilities import int_to_bits, float_to_bits
print(int_to_bits(57), float_to_bits(2.257))